## Introduction:
The aim of this document is to look at theatre in Warsaw and answer questions:

1) Is there any relation between rating of the theatre and number of venues around?

2) Is there any relation between popularity of the theatre and number of venues around? This issues can be important when you want to establish your own theatre.

## Data:
Data for this paper was downloaded from wikipedia. There is an article in polish about theatres in Warsaw and this article has table of places with adresses. Basic on this table using googlemaps specific latitude and longitude was collected. After that each location was scoped using foursquare in 500m around to find places in each location.

## Methodology
Collected data was cleaned, scoped, summarised in many ways to zoom the structure. It was also shown on the folium map where each theatre is located. After that linear regression models were performed to find answer to the questions.

## Results
I found no relation between rating and number of venues around. I found also no relation between popularity of ratings and venues around. It seems that popularity and rating of the teatre depends on diffrent factors (like actors, plays, director, etc.) which were not included in this research.

## Conclusions
Before final conclusion there i think it's worth to look for type of venues and rating and popularity (in fact this was the main purpose of this analysis). Unfortunately, foursquare classifies too many types of venues (around 200) and it can't be used to include 200 predictors in model with 50 observations.


In [3]:
conda install -c conda-forge google-api-python-client

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [ ]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
from matplotlib import colors as colors
import sklearn.cluster
import folium
import requests
from IPython.display import clear_output
import googlemaps

In [7]:
url_theatres = "https://pl.wikipedia.org/wiki/Teatry_w_Warszawie"
theatres = requests.get(url_theatres).text

soup = BeautifulSoup(theatres)
table = soup.find_all('table')[0]
t_df = pd.read_html(str(table))[0]
t_df.drop(columns = ['Unnamed: 5'], inplace=True)
t_df

,teatr,rodzaj,prowadzący,siedziba,www
0,Mazowiecki Teatr Muzyczny Operetka,teatr operetkowy,Włodzimierz Izban,"Bielany, ul. Kolumbijska 3",www.mteatr.pl
1,Nowy Teatr,teatr dramatyczny,Krzysztof Warlikowski,"Mokotów, ul. Madalińskiego 10/16",www.nowyteatr.org
2,Och-Teatr,teatr dramatyczny,Katarzyna Błachiewicz (dyrektor teatrów Fundac...,"Ochota, ul. Grójecka 65",www.ochteatr.com.pl
3,Stara ProchOFFnia,NaN,Wojciech Feliksiak,"Śródmieście, ul. Boleść 2",www.staraprochoffnia.scek.pl
4,Studio Buffo,teatr muzyczny,"Janusz Stokłosa (prezes), Janusz Józefowicz (d...","Śródmieście, ul. Konopnickiej 6",www.studiobuffo.com.pl
5,Studium Teatralne,NaN,NaN,"Praga-Południe, ul. Lubelska 30/32",www.studiumteatralne.pl
6,Teatr 6. piętro,teatr dramatyczny,Michał Żebrowski i Eugeniusz Korin,"Śródmieście, Pałac Kultury i Nauki – wejście o...",www.teatr6pietro.pl
7,Teatr Ateneum,teatr dramatyczny,Ryszard Markow,"Śródmieście, ul. Jaracza 2",www.teatrateneum.pl
8,Teatr Baj,teatr lalek,Ewa Piotrowska,"Praga-Północ, ul. Jagiellońska 28",www.teatrbaj.waw.pl
9,Teatr Bajka,teatr komediowy,Natalia Pietkiewicz-Szwarc,"Śródmieście, ul. Marszałkowska 138",www.teatrbajka.pl


In [8]:

t_df['rodzaj'].value_counts().to_frame(name="Count")

,Count
teatr dramatyczny,21
teatr komediowy,6
teatr muzyczny,3
teatr lalek,2
teatr operowy,2
teatr dziecięcy,1
teatr familijny,1
teatr niezależny,1
teatr operetkowy,1
teatr offowy,1
